# Retail Demo Store Experimentation Workshop - Amazon CloudWatch Evidently

In this workshop we will create an A/B experiment using [Amazon CloudWatch Evidently](https://aws.amazon.com/cloudwatch/features/). Evidently lets application developers conduct experiments and identify unintended consequences of new features before rolling them out for general use, thereby reducing risk related to new feature roll-out. Evidently allows you to validate new features across the full application stack before release, which makes for a safer release. When launching new features, you can expose them to a small user base, monitor key metrics such as page load times or conversions, and then dial up traffic. Evidently also allows you to try different designs, collect user data, and release the most effective design in production. 

Recommended Time: 30 minutes

## CloudWatch Evidently concepts

To get started with CloudWatch Evidently, for either a feature launch or an A/B experiment, you first create a project. A project is a logical grouping of resources. Within the project, you create features that have variations that you want to test or launch.

When the Retail Demo Store project was deployed in your account, an Evidently project and several Evidently features were automatically created. These resources are defined in the [evidently.yaml](https://github.com/aws-samples/retail-demo-store/blob/master/aws/cloudformation-templates/base/evidently.yaml) CloudFormation stack. Open a new browser window/tab and browse to [CloudWatch](https://console.aws.amazon.com/cloudwatch/home); under "Application monitoring" in the left navigation, you will find Evidently.

![Evidently Project page](./images/evidently/evidently_project.png)

### Retail Demo Store features

When a user is interacting with web application views like the home page, product detail page, the search auto-complete drop-down, and the shop livestreams page, the web application will make requests to the Recommendations microservice to retrieve or order products for the given user experience. One of the parameters passed with each of these requests is the feature name. The feature name is mapped to Evidently features. The following application features are already instrumented in the Retail Demo Store web application as well as mapped to Evidently features in the Evidently project.

- **Home page (top)**: the top of the home page view displays recommended products in two states: when the current visitor/user is new/cold and when the current visitor is warm. The new/cold visitor/user is automatically transitioned to the warm experience after 2 product view events have been sent to the Personalize event tracker. Selecting an existing shopper also puts the user into the warm user experience.
    - "**Popular products**": for new/cold users, displays popular products in a grid widget. The feature name is `home_product_recs_cold`.
    - "**Inspired by your shopping trends**": for existing/warm users, displays products from one of the supported product recommenders configured in the Recommendations microservice. The feature name is `home_product_recs`.
- **Home page (bottom)**: the bottom of the home page view displays featured products in a carousel widget.
    - "**Featured products**": this is the carousel widget at the bottom of the home page view where featured products are displayed. The feature name is `home_featured_rerank`.
- **Product detail page**: when you click on a product, you are taken to the product detail view.
    - "**Compare similar items**": this the carousel widget on the product detail view that displays products similar to the product being displayed. The feature name is `product_detail_related`.
- **Navigation**: the header navigation.
    - "**Search results**": this is the search drop-down in the web application's navigational header. For this feature, we can test personalized ranking of search results against search results that are ordered by Amazon OpenSearch. The feature name is `search_results`.
- **Shop live streams**: on the "Shop" drop-down is a "Shop Livestreams" option. This page provides a live streaming interface for demonstrating products and making recommendations.
    - "**Shop livestreams - discounted products**": this is the sidebar vertical widget on the Shop Livestreams page. It displays discounted products highighted in the live stream.  The feature name is `live_stream_prod_recommendation`.
    - "**Shop livestreams - Compare similar items**": this is the carousel widget at the bottom of the Shop Livestreams page. It displays products similar to the product currently being featured in the live stream. The feature name is `live_stream_prod_discounts`.

To examine the Evidently features, click on the Retail Demo Store's project in Evidently.

![Evidently Features page](./images/evidently/evidently_features.png)

## Retail Demo Store / CloudWatch Evidently integration

Let's examine the Evidently integration in more detail. 

- You start by creating features in Evidently mapped to user experiences in your application that you want to control with launch/feature flags or experiments. As mentioned above, this was already done for you in the Retail Demo Store during deployment. User experiences can be as simple as a view title or button color or as complex as a panel or widget.
- Next you instrument the user interface logic in your application for each feature to call Evidently to determine the variation to use. Each feature can have one or more variations and each variation has a value that can be a boolean/flag, number, or string. In our case, we're storing small JSON snippets as string variation values. The JSON contains the details needed to map a variation value to a recommender implementation.
- Experiments can be created for features by specifying the experiment duration, how to split traffic across variations, metrics that will be collected to measure the outcome of the experiment, and more.
- Your application calls the Evidently [EvaluateFeature](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_EvaluateFeature.html) or [BatchEvaluateFeature](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_BatchEvaluateFeature.html) API to retrieve the variation details for given feature(s) for the current user. This can be done in the client using the AWS SDK for JavaScript or server-side using any of the supported language SDKs. For the Retail Demo Store, these calls are made in the Recommendations microservice. 
   - If an experiment is active for a feature, the evaluate feature response will include a `reason` of `EXPERIMENT_RULE_MATCH`. In this case, you also need to send an experiment exposure event to Evidently to indicate that the current user is receiving the assigned variation.
- When/if the user "converts" against a variation that is part of an experiment, a "conversion" event needs to be sent to Evidently. In our case, a conversion is when the user clicks on a product included in a variation's recommended products. The Retail Demo Store web application calls the `/experiment/outcome` endpoint on the Recommendations microservice which then calls the Evidently [PutProjectEvents](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_PutProjectEvents.html) API.

The following architecture diagram summarizes the integration.

![Retail Demo Store Evidently architecture](./images/evidently/rds_evidently_architecture.png)


## Running an Evidently experiment

Let's walk through the process of creating an experiment in Evidently, examine how the Retail Demo Store web application displays variations, and then simulate an experiment so we can inspect the results. We're going to illustrate how to run an experiment in code using the Python programming language but you could also use the Evidently console.

### Our experiment hypothesis

**Sample scenario:**

Website analytics have shown that user sessions frequently end on the home page for our e-commerce site, the Retail Demo Store. Furthermore, when users do make a purchase, most purchases are for a single product. Currently on our home page we are using a basic approach of recommending featured products (control experience). We hypothesize that replacing the current featured products approach on the homepage with personalized recommendations from Amazon Personalize (variation) will result in increasing the CTR (click-through rate) of products by 25%. The current click-through rate is 15%.

### Setup - import dependencies

First, let's import the dependencies needed to interact with the Evidently API via Python.

In [ ]:
import boto3
import json
import requests
import time
import pandas as pd
import numpy as np
import random
import scipy.stats as scs
from collections import defaultdict
from datetime import datetime, timedelta

Next let's create the clients we'll need to make API calls. We'll also declare the feature name that we'll be using for this experiment (`home_product_recs`).

In [ ]:
# Evidently client
evidently = boto3.client('evidently')
# Service discovery will allow us to dynamically discover Retail Demo Store microservices
servicediscovery = boto3.client('servicediscovery')

feature = 'home_product_recs'

# The Uid is a unique ID and we need it to find the role made by CloudFormation
with open('/opt/ml/metadata/resource-metadata.json') as f:
    data = json.load(f)
sagemaker = boto3.client('sagemaker')
sagemakerResponce = sagemaker.list_tags(ResourceArn=data["ResourceArn"])
for tag in sagemakerResponce["Tags"]:
    if tag['Key'] == 'Uid':
        project_name = tag['Value']
        break

print('project_name:', project_name)

### Inspect feature

Before creating and starting our experiment, let's take a look at the feature. As a reminder, this feature was setup in Evidently during the deployment of the Retail Demo Store.

In [ ]:
response = evidently.get_feature(project = project_name, feature = feature)
print(json.dumps(response['feature'], indent = 2, default = str))

In the response above, take special note of the `variations` array and the `value.stringValue` value for each variation. As mentioned above, we're actually storing a short JSON document as the `stringValue`. The Recommendations microservice will parse this string as JSON and extract the information it needs to "wire up" each variation to a product recommendation implementation. In this case we have two variations for this feature:

- Variation 0: **Featured Products** - displays featured products from the product catalog. For this variation, we just need the `type` of `product` to tell the Recommendations service how to resolve featured products from the Products microservice.
- Variation 1: **Personalize-UserPersonalization** - displays personalized product recommendations from an Amazon Personalize campaign or recommender. To resolve this variation we need a `type` of `personalize-recommendations` and the Personalize campaign or recommender ARN to call to get recommendations for each user. The `arn` field in the JSON snippet provides the ARN needed by the Recommendations service.

### Evaluate feature before experiment

Before creating an experiment, let's take a look at what the [EvaluateFeature](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_EvaluateFeature.html) API returns for the `home_product_recs` feature.

In [ ]:
user_id = '123'

response = evidently.evaluate_feature(
    entityId = user_id,
    feature = feature,
    project = project_name
)

print(json.dumps(response, indent = 2, default = str))

Take note that the `reason` is `DEFAULT` and the only variation and value is the default variation from the `GetFeature` call above. This means that there is currently not an active experiment or launch for this feature.

### Create experiment

Now we will create an experiment for the `home_product_recs` feature by calling the [CreateExperiment](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_CreateExperiment.html) API. You could also create the experiment in the Evidently console.

First we will define the metric event pattern. This pattern is used match events for exposures and outcome/conversion events across all features back to this particular experiment. This is very similar to Amazon EventBridge [patterns](https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-event-patterns.html). The logic in the Recommendations service is to use `userDetails.userId` to identify the user and to build the `valueKey` by taking the feature name, convert it from snake case to camel case, and append `Clicked` to the result. So `home_product_recs` becomes `homeProductRecsClicked`. This is put within a `details` dictionary to arrive at a final value key of `details.homeProductRecsClicked`. Experiments for every feature in the Retail Demo Store is handled the same way.

To hopefully pull this together, here is an example of a conversion event that the Recommendations service will send to Evidently for user `abc123`. Since we're tracking click-through-rate, the value is `1.0` to indicate a conversion/click for this user.

```javascript
{
    'details': {
        'homeProductRecsClicked': 1.0
    },
    'userDetails': {
        'userId': 'abc123'
    }
}
```

Now the pattern that Evidently needs to match events formatted like this is defined in the following cell.

In [ ]:
metric_event_pattern = {
    "userDetails.userId": [
        {
            "exists": True
        }
    ],
    "details.homeProductRecsClicked": [
        {
            "exists": True
        }
    ]
}

We will also define the variations to use for the experiment.

In [ ]:
experiment_variations = [
    {
        'feature': feature,
        'variation': 'FeaturedProducts',
        'name': 'FeaturedProducts'
    },
    {
        'feature': feature,
        'variation': 'Personalize-UserPersonalization',
        'name': 'Personalize-UserPersonalization'
    }
]

Now we can create the experiment, passing the metric event pattern as the `metricGoals` argument and the experiment variations as the `treatments` argument. The control treatment will be `FeaturedProducts` (the "A" in our A/B test), the default non-personalized product recommendations on the homepage. The `Personalize-UserPersonalization` variation will be the "B" in our A/B test.

In [ ]:
experiment_name = 'home_product_recs_personalization'

response = evidently.create_experiment(
    project = project_name,
    name = experiment_name,
    description = 'Testing default product recommendations of featured products against Amazon Personalize generated recommendations',
    metricGoals = [
        {
            'desiredChange': 'INCREASE',
            'metricDefinition': {
                'entityIdKey': 'userDetails.userId',
                'valueKey': 'details.homeProductRecsClicked',
                'eventPattern': json.dumps(metric_event_pattern),
                'name': 'homeProductRecsClicked'
            }
        },
    ],
    onlineAbConfig={
        'controlTreatmentName': 'FeaturedProducts',
        'treatmentWeights': {
            'FeaturedProducts': 50000,
            'Personalize-UserPersonalization': 50000
        }
    },
    samplingRate = 100000,
    treatments = experiment_variations
)

print(json.dumps(response['experiment'], indent = 2, default = str))

### Start experiment

With the experiment created, now it's time to start the experiment to make it active. We'll set the experiment end date to be one week from now.

In [ ]:
experiment_end_date = datetime.now() + timedelta(days = 7)

response = evidently.start_experiment(
    project = project_name,
    experiment = experiment_name,
    analysisCompleteTime = experiment_end_date
)

print(json.dumps(response, indent = 2, default = str))

As mentioned earlier, you could have completed the previous steps in the CloudWatch Evidently console as well.

### Evaluate feature after experiment has started

Now let's take a look at the EvaluateFeature API response now that the experiment has been created and started.

In [ ]:
response = evidently.evaluate_feature(
    entityId = user_id,
    feature = feature,
    project = project_name
)

print(json.dumps(response, indent = 2, default = str))

Notice this time that the `reason` is now `EXPERIMENT_RULE_MATCH`, we have details on the active experiment in the `details` dictionary, and we have details on the assigned variation in the `value` dictionary. The Recommendations service uses this response to know which resolver to use to provide the response.

### Inspect storefront

To see the experiment in action on the Retail Demo Store storefront, browse to the storefront deployed in your account, and then either sign in or create an account. On the home view you should see an indicator that an experiment is active and the variation assigned to the currently selected shopper is indicated for each product.

_Note: the Recommendations service caches `BatchEvaluateFeature` responses so you may not see feature and experiment changes reflected in the storefront for up to 30 seconds._ 

![Retail Demo Store active Evidently experiment](./images/evidently/rds_active_evidently_experiment.png)

To see how the assigned variation changes depending on the current shopper, try switching shoppers from the "Shopper" dropdown in the top naviation.

![Retail Demo Store switch shopper](./images/evidently/rds_switch_shoppers.png)

You may have to cycle through a few shoppers to land on one that hashes to a different variation.

#### Recommendations service implementation details

Diving deeper into the `BatchEvaluateFeature` implementation in the Recommendations service, the [EvidentlyFeatureResolver](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/evidently_feature_resolver.py) class has a function named `_call_evidently_evaluate_features`. This function is called to evaluate all of the known features in the Retail Demo Store for the current user. This is more efficient than calling `EvaluateFeature` for each feature and allows us to cache the response in the service. Below is the snippet of the relevant source code.

```python
class EvidentlyFeatureResolver:
    def _call_evidently_evaluate_features(self, user_id: str) -> List[Dict]:
        requests = []
        for feature in FEATURE_NAMES:
            requests.append({
                'entityId': user_id,
                'feature': feature
            })

        response = evidently.batch_evaluate_feature(
            project=project_name,
            requests=requests
        )

        return response['results']
```

### Create conversion/click events

To trigger a conversion event to be sent to Evidently's [PutProjectEvents](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_PutProjectEvents.html) API, click on a product displayed by the variation. As you switch shoppers, trigger conversion events for some and not others to see how they're tracked in the Evidently results dashboard for the experiment. You can also inspect the requests sent from the Retail Demo Store web application to the Recommendations service using the Developer Tools in your browser. The two endpoints are `/recommendations` to retrieve recommendations and `/experiment/outcome` to send outcome notifications to the backend.

#### Recommendations service implementation details

Diving deeper into the `PutProjectEvents` implementation in the Recommendations service, the [EvidentlyExperiment](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/experiment_evidently.py) class has a function named `_send_evidently_event`. This function is called to send exposure and conversion events for a user and metric when the `/experiment/outcome` endpoint is called from the web application. Below is the snippet of the relevant source code.

```python
class EvidentlyExperiment(experiment.Experiment):
    def _send_evidently_event(self, user_id: str, metric_value: float, timestamp: datetime = datetime.now()):
        # In case None is passed for timestamp
        timestamp = datetime.now() if not timestamp else timestamp

        # We convert the feature name from snake case to camel case for the metric value key.
        metric_name = f'{self._snake_to_camel_case(self.feature)}Clicked'

        response = evidently.put_project_events(
            project = self.project,
            events = [
                {
                    'type': 'aws.evidently.custom',
                    'timestamp': timestamp,
                    'data': json.dumps({
                        'details': {
                            metric_name: metric_value
                        },
                        'userDetails': {
                            'userId': str(user_id)
                        }
                    })
                }
            ]
        )
```

## Simulate experiment

Now let's simulate a running experiment by calling the `/recommendations` and `/experiment/outcome` endpoints on the Recommendations microservice for samples of users from the Users microservice. The following code cells will prepare the data and functions needed to run the simulation as well as show the results.

### Load Users

For our experiment simulation, we will load all Retail Demo Store users and run the experiment until the sample size for both variations has been met.

First, let's discover the IP address for the Retail Demo Store's [Users](https://github.com/aws-samples/retail-demo-store/tree/master/src/users) service.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='users',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

users_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Users Service Instance IP: {}'.format(users_service_instance))

Next, let's fetch all users, randomize their order, and load them into a local data frame.

In [ ]:
# Load all users so we have enough to satisfy our sample size requirements.
response = requests.get('http://{}/users/all?count=10000'.format(users_service_instance))
users = response.json()
random.shuffle(users)
users_df = pd.DataFrame(users)
pd.set_option('display.max_rows', 5)

users_df

### Discover Recommendations service

Next, let's discover the IP address for the Retail Demo Store's [Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) service. This is the service where the Experimentation framework and Evidently integration is implemented. We will call endpoints on this service to simulate our Evidently A/B experiment.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='recommendations',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

recommendations_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Recommendation Service Instance IP: {}'.format(recommendations_service_instance))

### Sample Size Calculation

The first step is to determine the sample size necessary to reach a statistically significant result given a target of 25% gain in click-through rate from the home page. There are several sample size calculators available online including calculators from [Optimizely](https://www.optimizely.com/sample-size-calculator/?conversion=15&effect=20&significance=95), [AB Tasty](https://www.abtasty.com/sample-size-calculator/), and [Evan Miller](https://www.evanmiller.org/ab-testing/sample-size.html#!15;80;5;25;1). For this exercise, we will use the following function to calculate the minimal sample size for each variation.

In [ ]:
def min_sample_size(bcr, mde, power=0.8, sig_level=0.05):
    """Returns the minimum sample size to set up a split test

    Arguments:
        bcr (float): probability of success for control, sometimes
        referred to as baseline conversion rate

        mde (float): minimum change in measurement between control
        group and test group if alternative hypothesis is true, sometimes
        referred to as minimum detectable effect

        power (float): probability of rejecting the null hypothesis when the
        null hypothesis is false, typically 0.8

        sig_level (float): significance level often denoted as alpha,
        typically 0.05

    Returns:
        min_N: minimum sample size (float)

    References:
        Stanford lecture on sample sizes
        http://statweb.stanford.edu/~susan/courses/s141/hopower.pdf
    """
    # standard normal distribution to determine z-values
    standard_norm = scs.norm(0, 1)

    # find Z_beta from desired power
    Z_beta = standard_norm.ppf(power)

    # find Z_alpha
    Z_alpha = standard_norm.ppf(1-sig_level/2)

    # average of probabilities from both groups
    pooled_prob = (bcr + bcr+mde) / 2

    min_N = (2 * pooled_prob * (1 - pooled_prob) * (Z_beta + Z_alpha)**2
             / mde**2)

    return min_N

Next we will call the `min_sample_size` function with the parameters of our testing hypothesis.

In [ ]:
# This is the ficticious conversion rate of the existing control experience
baseline_conversion_rate = 0.15
# This is the lift expected by adding personalization with our variation
absolute_percent_lift = baseline_conversion_rate * .25

# Calculate the sample size needed to reach a statistically significant result
sample_size = int(min_sample_size(baseline_conversion_rate, absolute_percent_lift))

print('Sample size for each variation: ' + str(sample_size))

### Simulation Function

The following `simulate_experiment` function is supplied with the sample size for each group (A and B) and the probability of conversion for each group that we want to use for our simulation. It runs across enough users to satisfy the sample size requirements and calls the Recommendations service for each user in the experiment.

In [ ]:
def simulate_experiment(N_A, N_B, p_A, p_B):
    """Returns a pandas dataframe with simulated CTR data

    Parameters:
        N_A (int): sample size for control group
        N_B (int): sample size for test group
            Note: final sample size may not match N_A & N_B provided because the
            group at each row is chosen at random by the ABExperiment class.
        p_A (float): conversion rate; conversion rate of control group
        p_B (float): conversion rate; conversion rate of test group

    Returns:
        df (df)
    """

    # will hold exposure/outcome data
    data = []

    # total number of users to sample for both variations
    N = N_A + N_B
    
    if N > len(users):
        raise ValueError('Sample size is greater than number of users')

    print(f'Generating data for {N} users... this may take a few minutes')
    print('While waiting for the simulation to finish, open the CloudWatch Evidently console in another browser tab/window to view results')

    # initiate bernoulli distributions to randomly sample from based on simulated probabilities
    A_bern = scs.bernoulli(p_A)
    B_bern = scs.bernoulli(p_B)
        
    outcome_url = f'http://{recommendations_service_instance}/experiment/outcome'
    
    recs_responses = defaultdict(int)
    outcome_responses = defaultdict(int)
    
    for idx in range(N):
        if idx > 0 and idx % 500 == 0:
            print('Generated data for {} users so far'.format(idx))
            
        # initite empty row
        row = {}

        # Get next user from shuffled list
        user = users[idx]

        # Call Recommendations web service to get recommendations for the user
        rec_url = f'http://{recommendations_service_instance}/recommendations?userID={user["id"]}&feature={feature}'
        response = requests.get(rec_url)
        recs_responses[str(response.status_code)] += 1

        recommendations = response.json()
        recommendation = recommendations[random.randint(0, len(recommendations)-1)]
        
        variation = recommendation['experiment']['variationIndex']
        row['variation'] = variation
        
        # Determine if variation converts based on probabilities provided
        if variation == 'FeaturedProducts':
            row['converted'] = A_bern.rvs()
        elif variation == 'Personalize-UserPersonalization':
            row['converted'] = B_bern.rvs()
            
        if row['converted'] == 1:
            # Update experiment with outcome/conversion
            payload = {
                'correlationId': recommendation['experiment']['correlationId']
            }
            response = requests.post(outcome_url, data=payload)
            time.sleep(.150)
            outcome_responses[str(response.status_code)] += 1
        
        data.append(row)
        
        time.sleep(.150)
        
    # convert data into dataframe
    df = pd.DataFrame(data)
    
    print('Done')
    
    print("Recommendations response stats:")
    print(recs_responses)
    print("Outcome response stats:")
    print(outcome_responses)

    return df

### Run Simulation

Next we run the simulation by defining our simulation parameters for sample sizes and probabilities and then call `simulate_experiment`. This will take 15-20 minutes depending on the sample sizes. Open the Evidently results page in the CloudWatch Evidently console in another browser tab/window while the simulation runs to view the results in real-time. 

_Note: you may need to occasionally hard-refresh the Evidently results console page to see updated results as the simulation code below runs._

In [ ]:
%%time

# Set size of both groups to calculated sample size
N_A = N_B = sample_size

# Use probabilities from our hypothesis
# bcr: baseline conversion rate
p_A = 0.15
# d_hat: difference in a metric between the two groups, sometimes referred to as minimal detectable effect or lift depending on the context
p_B = 0.1875

# Run simulation
ab_data = simulate_experiment(N_A, N_B, p_A, p_B)

ab_data

### Analyze simulation results

Next, let's take a closer look at the results of our simulation. We'll start by calculating some summary statistics.

In [ ]:
ab_summary = ab_data.pivot_table(values='converted', index='variation', aggfunc=np.sum)
# add additional columns to the pivot table
ab_summary['total'] = ab_data.pivot_table(values='converted', index='variation', aggfunc=lambda x: len(x))
ab_summary['rate'] = ab_data.pivot_table(values='converted', index='variation')
ab_summary

### Review Evidently results

Once the simulation is complete, open (or refresh) the Evidently experiment results page in the CloudWatch console. The first view  summarizes the event counts. This just tells us the volume of exposure and conversion events for each variation.

![Evidently experiment event counts](./images/evidently/evidently_results_event_counts.png)

Change the view to summarize on average values. This will reflect the CTR conversion events for each variation. You should see that the `Personalize-UserPersonalization` variation is the `Better` variation for this experiment. 

_Note: sometimes the colors used in the graph do not match the colors in the table below the graph for each variation. Refer to the graph legend._

![Evidently experiment average value](./images/evidently/evidently_results_avg_value.png)

### Stop experiment

With the simulation finished, we will go ahead and stop the experiment. We will set the `desiredState` to `CANCELLED` (`COMPLETED` is the other possible value). By stopping the experiment, the Recommendations service will resume serving the default home page personalization user experience. It also allows other experiment types in the other Experimentation workshops to be tested.

In [ ]:
response = evidently.stop_experiment(
    desiredState = 'CANCELLED',
    experiment = experiment_name,
    project = project_name,
    reason = 'Completed experiment testing in the Evidently workshop'
)

print(json.dumps(response, indent = 2, default = str))

## Next Steps

You have completed the exercise for implementing an A/B test using the Amazon CloudWatch Evidently. In this workshop we only tested the `home_product_recs` feature on the homepage. There are several other features in the web application (listed near the top of this page) that are also instrumented and have Evidently features configured. Try setting up one or more experiments for the other features.

Since Evidently features are evaluated before the built-in experimentation types, **be sure to cancel your Evidently experiment**. This can be done in the code cell above or in the CloudWatch > Evidently console.